In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
import torch
from torch.utils.data import Dataset
import numpy as np
import os

df = pd.read_csv('dataset.csv')

# Handle class imbalance
df_majority = df[df.Genre == 'Drama']
df_minority = df[df.Genre != 'Drama']
df_majority_downsampled = resample(df_majority, replace=False, n_samples=200, random_state=42)
df_balanced = pd.concat([df_majority_downsampled, df_minority])
genre_counts = df_balanced['Genre'].value_counts()
df_filtered = df_balanced[df_balanced['Genre'].isin(genre_counts[genre_counts >= 10].index)]

class MovieDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')
model = BertForSequenceClassification.from_pretrained('HooshvareLab/bert-fa-base-uncased', num_labels=len(genre_counts[genre_counts >= 10]))

# Encode labels
label_encoder = LabelEncoder()
df_filtered['Genre'] = label_encoder.fit_transform(df_filtered['Genre'])

# Define cross-validation
skf = StratifiedKFold(n_splits=5)

# Early Stopping callback
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=3):
        self.early_stopping_patience = early_stopping_patience
        self.best_loss = None
        self.patience_counter = 0

    def on_evaluate(self, args, state, control, **kwargs):
        logs = kwargs['metrics']
        if self.best_loss is None or logs['eval_loss'] < self.best_loss:
            self.best_loss = logs['eval_loss']
            self.patience_counter = 0
        else:
            self.patience_counter += 1

        if self.patience_counter >= self.early_stopping_patience:
            print("Stopping early")
            control.should_training_stop = True

def train_and_evaluate(train_index, test_index, fold_num, device):
    train_df = df_filtered.iloc[train_index]
    test_df = df_filtered.iloc[test_index]

    train_dataset = MovieDataset(
        texts=train_df['Content_1'].tolist(),
        labels=train_df['Genre'].tolist(),
        tokenizer=tokenizer,
        max_len=128
    )

    test_dataset = MovieDataset(
        texts=test_df['Content_1'].tolist(),
        labels=test_df['Genre'].tolist(),
        tokenizer=tokenizer,
        max_len=128
    )

    # Define the training arguments
    training_args = TrainingArguments(
        output_dir=f'./results/fold_{fold_num}',
        num_train_epochs=10,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs/fold_{fold_num}',
        logging_steps=10,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        learning_rate=5e-5,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        report_to='none',
        no_cuda=False if torch.cuda.is_available() else True
    )

    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model on the test set
    results = trainer.evaluate(test_dataset)

    # Predict on the test set
    predictions, labels, _ = trainer.predict(test_dataset)
    predictions = predictions.argmax(axis=1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, predictions)

    # Calculate F1 score
    f1 = f1_score(labels, predictions, average='macro')

    print(f'Fold {fold_num} Accuracy: {accuracy:.4f}')
    print(f'Fold {fold_num} F1 Score: {f1:.4f}')
    print(classification_report(labels, predictions, target_names=label_encoder.classes_))

    return accuracy, f1

# Cross-validation loop
accuracies = []
f1_scores = []

# Assuming 4 GPUs
num_gpus = 4

for fold_num, (train_index, test_index) in enumerate(skf.split(df_filtered, df_filtered['Genre'])):
    device = f'cuda:{fold_num % num_gpus}' if torch.cuda.is_available() else 'cpu'
    os.environ["CUDA_VISIBLE_DEVICES"] = str(fold_num % num_gpus)
    
    accuracy, f1 = train_and_evaluate(train_index, test_index, fold_num, device)
    accuracies.append(accuracy)
    f1_scores.append(f1)

# Print overall results
print(f'Overall Accuracy: {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}')
print(f'Overall F1 Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/k8/xdc9g2pn7v561ty2fk116vjh0000gn/T/ipykernel_6271/317131113.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Genre'] = label_encoder.fit_transform(df_filtered['Genre'])
/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnin

{'loss': 2.4798, 'grad_norm': 8.154491424560547, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.24}


  5%|▍         | 20/420 [00:37<12:15,  1.84s/it]

{'loss': 2.4076, 'grad_norm': 9.875574111938477, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.48}


  7%|▋         | 30/420 [00:55<11:49,  1.82s/it]

...

100%|██████████| 11/11 [00:04<00:00,  2.27it/s]


Stopping early


100%|██████████| 11/11 [00:04<00:00,  2.26it/s]

Fold 4 Accuracy: 0.8563
Fold 4 F1 Score: 0.7340
                          precision    recall  f1-score   support

                  Action       1.00      1.00      1.00        16
               Adventure       1.00      1.00      1.00         9
       Arts & Literature       1.00      0.14      0.25         7
                  Comedy       1.00      0.94      0.97        54
                   Crime       1.00      0.92      0.96        13
    Culture & Traditions       0.00      0.00      0.00         4
                   Drama       0.62      1.00      0.77        40
                  Family       1.00      1.00      1.00         5
                 History       1.00      0.75      0.86         4
Human Interest & Society       0.00      0.00      0.00         9
                 Romance       1.00      1.00      1.00         2
                     War       1.00      1.00      1.00         4

                accuracy                           0.86       167
               macro avg  


/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mohammadmahdi/Documents/test/nlp/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{